### Reporting on Attachments from Salesforce

For whatever reason Salesforce's standard reporting tool doesn't allow direct reporting from the Attachments object. Below is a simple example script to produce such a report using the simple_salesforce Python library.


In [1]:
# required libraries
import time
import pandas as pd
from simple_salesforce import Salesforce

# Replace the xxx's with your own Salesforce org and login details
sf = Salesforce(password='xxxxxxxxxxxxxxxxxxxxxxxxx', \
                username='xxxxxxxxxxxxxxxxxxxxxxxxx', \
                organizationId='xxxxxxxxxxxxxxxxxxxxxxxxx',\
                security_token='xxxxxxxxxxxxxxxxxxxxxxxxx')

In this example I'm going to get all attachments from Salesforce with the word 'key' in the title.

In [2]:
# Get the Attachments with a SOQL query
query= """
SELECT Name, CreatedDate, OwnerId
FROM Attachment
WHERE Name LIKE '%key%'

"""

result = sf.query_all(query)
Attachment_df = pd.DataFrame(result['records']) # Stores the results in a Pandas dataframe
Attachment_df.drop('attributes',axis=1,inplace=True) # Removes 'attributes' column which is not required
Attachment_df.rename(columns={'Id':'AttachmentId','Name':'Attachment_Name'},inplace=True) # Renames the columns
Attachment_df.head()  # Have a look at the first few rows to check it worked

,CreatedDate,Attachment_Name,OwnerId
0,2017-01-12T09:42:12.000+0000,A.01.02 Key waiver.pdf,005b0000004pOvAAAU
1,2016-12-16T09:37:02.000+0000,A03.06 Key Release Permission Form.pdf,005b0000003ZalDAAS
2,2017-01-12T16:15:21.000+0000,B.08.03 Key Release Permission Form.pdf,005b0000001NLv1AAG
3,2017-01-11T12:07:54.000+0000,H.03.03 Key Waiver.pdf,005b0000004pOvAAAU
4,2016-02-25T16:20:48.000+0000,Key Facts Phase 3a.pdf,005b0000001NLuvAAG


Having Salesforce IDs in the above is not particularly helpful if the report is to be read by a human being; so we can run reports on other objects and merge in the useful details. As an example I'll use 'OwnerId' to merge in the name of the user who added the attachment.

In [3]:
# Get Users
query= """
SELECT Id, Name
FROM User
"""
result = sf.query_all(query)
User_df = pd.DataFrame(result['records'])
User_df.rename(columns={'Name':'User_Name','Id':'UserId'},inplace=True)
User_df.drop('attributes',axis=1,inplace=True)
User_df.head()

,UserId,User_Name
0,005b0000000wHPDAA2,James Smith
1,005b0000000wHPIAA2,Rob Jones
2,005b0000001ehRgAAI,Rebecca Cobby
3,005b0000001fTUJAA2,Maeve Barry
4,005b0000003YlQYAA0,Test Estates User


Finally we can merge the 2 dataframes to produce a final report.

In [4]:
final_report = pd.merge(Attachment_df,User_df,left_on='OwnerId',right_on='UserId',how='left')
final_report.drop(['OwnerId','UserId'],axis=1,inplace=True)
final_report.head()

,CreatedDate,Attachment_Name,User_Name
0,2017-01-12T09:42:12.000+0000,A.01.02 Key waiver.pdf,Ellie Williams
1,2016-12-16T09:37:02.000+0000,A03.06 Key Release Permission Form.pdf,Rebecca Derbyshire
2,2017-01-12T16:15:21.000+0000,B.08.03 Key Release Permission Form.pdf,Mark Barclay
3,2017-01-11T12:07:54.000+0000,H.03.03 Key Waiver.pdf,Ellie Williams
4,2016-02-25T16:20:48.000+0000,Key Facts Phase 3a.pdf,Sarah Gorman


The report can then be easily exported to CSV as you would a report from within Salesforce.

In [5]:
# Create a time-stamped file name for the report
file_timestamp = time.strftime("%Y_%m_%d_%H_%M")
filename = 'attachments_report '+file_timestamp+'.csv' 

# Create the CSV.
final_report.to_csv(filename)